# Q1

In [48]:
import numpy as np
import pandas as pd
import random
import matplotlib.pyplot as plt

In [49]:
train_data =  pd.read_csv("Train_Data.csv")
train_label =  pd.read_csv("Train_Labels.csv")
test_data =  pd.read_csv("Test_Data.csv")
test_label =  pd.read_csv("Test_Labels.csv")

### Kmeans (from scratch)

In [17]:
def K_mean(df,k):
    print('is loading . . . ')
    
    global_cost = []
    global_kmean = []
    labels = []
    #remove duplicate data
    datas = df.drop_duplicates()
    index = list(datas.index.values.astype(int))
    
    for i in range(1000):
        
        data = datas.copy()
        centroids = []
        #initialize centroids
        indexofcenteroids = random.sample(index, k)
        for i in indexofcenteroids:
            centroids.append(np.array(data.loc[i,:]))
        boolean=True 
        costs = [-1]
        while(boolean==True):                                  
            #find closest centroid cluster
            Kcenters = np.zeros([data.shape[0],3])
            cnt = 0
            for c in centroids:
                Kcenters[:,cnt] = np.sqrt(((data.iloc[:,0:13] - c)**2).sum(axis=1))
                cnt += 1

            L = np.array(Kcenters).argmin(axis = 1)
            data['cluster'] = L
            
            #move centroids
            clusters = list(data['cluster'].unique())
            clusters.sort()   
            kmeans=[]
            for c in clusters:
                C = data[data['cluster'] == c]
                means = C.iloc[:,0:13].mean(axis=0)
                kmeans.append(np.array(means))
        
            centroids = np.array(kmeans)
            

            #compute cost
            cost = 0
            for j in range(len(kmeans)):
                cluster = data[data['cluster'] == j]
                cost += (np.sqrt(((cluster.iloc[:,0:13] - kmeans[j])**2).sum(axis=1))).sum()
            #check convergence 
            if(costs.pop() == cost):
                global_kmean.append(kmeans)
                labels.append(L)
                global_cost.append(cost)
                boolean = False
            else:
                costs.append(cost)
            
        
    bestcost = min(global_cost)
    bestkmean = global_kmean[global_cost.index(min(global_cost))]
    bestcluster = labels[global_cost.index(min(global_cost))]
    return bestcost, bestkmean, bestcluster

### Kmeans for predicted (Test Data)

In [18]:
def kmean_predicted(data, centroids, label):
    
    data['label'] = label    
    Kcenters = np.zeros([data.shape[0],3])
    cnt = 0
    for c in centroids:
        Kcenters[:,cnt] = np.sqrt(((data.iloc[:,0:13] - np.array(c))**2).sum(axis=1))
        cnt += 1

    L = np.array(Kcenters).argmin(axis = 1) + 1
    data['cluster'] = L
    

### Confusion Matrix

In [55]:
def Confusion_Matrix(true_label, predicted_label):
    
    K = len(np.unique(true)) # Number of classes  
    cm = np.zeros([k,k])
    for i in range(len(true)):
        cm[true_label[i]-1][predicted_label[i]-1] += 1
    
                
    classes = ['1','2','3']
    total_classes = len(classes)
    level_labels = [total_classes*[0], list(range(total_classes))]

    cm_frame1 = pd.DataFrame(data=cm,columns=pd.MultiIndex(levels=[['Predicted_label:'],\
                                                                  classes],codes=level_labels),\
                        index=pd.MultiIndex(levels=[['True_label:'], classes], codes=level_labels))
    display(cm_frame1)

### Train Data:

In [19]:
cost,kmean,labels = K_mean(train_data,3)

is loading . . . 


In [20]:
print("cost value:",cost)

cost value: 9959.260510624143


In [21]:
for i in range(len(kmean)):
    print(f'center for cluster {i+1}:')
    display(kmean[i])

center for cluster 1:


array([1.25717021e+01, 2.64787234e+00, 2.27787234e+00, 2.04489362e+01,
       9.22127660e+01, 2.00978723e+00, 1.69872340e+00, 3.94893617e-01,
       1.42063830e+00, 4.21574468e+00, 9.26170213e-01, 2.42765957e+00,
       4.66595745e+02])

center for cluster 2:


array([1.28481081e+01, 2.43135135e+00, 2.39351351e+00, 1.98729730e+01,
       9.97297297e+01, 2.08756757e+00, 1.61648649e+00, 4.14864865e-01,
       1.45918919e+00, 5.44594595e+00, 8.92324324e-01, 2.43000000e+00,
       7.36810811e+02])

center for cluster 3:


array([1.38652e+01, 1.83480e+00, 2.40560e+00, 1.74120e+01, 1.06040e+02,
       2.84200e+00, 3.04120e+00, 2.84400e-01, 1.97520e+00, 5.53920e+00,
       1.08400e+00, 3.10200e+00, 1.17448e+03])

In [22]:
train_data['label'] = train_label
train = train_data.drop_duplicates()
display(pd.DataFrame(data={'predicted labels':labels, 'true labels':train['label']}))

,predicted labels,true labels
0,2,1
1,1,3
2,1,2
3,0,2
4,0,2
...,...,...
138,1,3
140,0,2
141,1,3
142,1,3


##### Note that here the actual tags and cluster names do not need to be the same, as your naming may be different, but here each cluster name must go to a specific tag (mapping)
0 ==== > 2

1 ==== > 3

2 ==== > 1

### Test Data:

In [50]:
centers = [kmean[2],kmean[0],kmean[1]]
kmean_predicted(test_data, centers, test_label)

### Confusion Matrix:

In [56]:
Confusion_Matrix(test_data['label'], test_data['cluster'])

Predicted_label:          
                             1    2    3
True_label: 1              7.0  0.0  1.0
            2              0.0  8.0  2.0
            3              0.0  5.0  8.0

### scikit-learn (kmeans)

In [74]:
from sklearn.cluster import KMeans

train_data =  pd.read_csv("Train_Data.csv")
train_label =  pd.read_csv("Train_Labels.csv")

test_data =  pd.read_csv("Test_Data.csv")
test_label =  pd.read_csv("Test_Labels.csv")
test_label[test_label['1'] != 3] = test_label[test_label['1'] != 3]%2+1

kmeans = KMeans(n_clusters=3, random_state=0).fit(train_data)
predicted_labels = kmeans.predict(test_data) +1
Confusion_Matrix(np.array(test_label), predicted_labels)


Predicted_label:          
                             1    2    3
True_label: 1              8.0  0.0  2.0
            2              0.0  7.0  1.0
            3              5.0  0.0  7.0

Sharare Zolghadr (610395109)